In [1]:
import os
import numpy as np
#os.environ['AUTOGRAPH_VERBOSITY'] = '0'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

In [2]:
print(tf.__version__)

2.5.0


In [3]:
import pickle
with open('/root/autodl-nas/workspace/datasets/ml/ml-100k/all_ratings.pkl', 'rb') as f:  
    user_dict = pickle.load(f)
items_num=[]
for i in user_dict.values():
    items_num.append(max(i))
items_num=max(items_num)

maxlen=124
len_seq=10

batch_size=512
epoch_num=100
hidden_size=64
keep_rate=0.9
layers_num=3
num_heads=1
interest_num=1
kernel_size=4
transformerencoder=0
multi_dense=0
neg_num=5
test_neg_num=1000

In [4]:
import random
import numpy as np
def sample(user_dict,maxlen,len_seq):
    train_set=[]
    train_val_set=[]
    val_set=[]
    test_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        hist=hist[-maxlen:-2]
        #print(hist)
        for i in range(1,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #print(hist[0:i])
            seq[max(0,len_seq+idx-1):]=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            train_set.append((u,list(seq),nxt))
            #print(seq)
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        test_set.append((u,list(seq),nxt))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set,val_set,train_val_set
            
def non_zero_sample(user_dict,maxlen,len_seq):
    train_set=[]
    test_set=[]
    val_set=[]
    train_val_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        seq = np.zeros([len_seq], dtype=np.int32)
        #print(np.shape(seq[-maxlen:]))
        while(len(hist)<maxlen):
            hist.insert(0,0)
        #print(len(hist),hist)
        hist=hist[-maxlen:-2]
        for i in range(len_seq,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #rint(hist[max(0,i-len_seq):i])
            seq=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
            train_set.append((u,list(seq),nxt,neg_item))
            #print(seq)
        #print(np.shape(hist[0:len(hist)-1]))
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
        val_set.append((u,list(seq),nxt,neg_item))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        neg_item = [random.randint(1, items_num) for _ in range(test_neg_num)]
        test_set.append((u,list(seq),nxt,neg_item))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set ,val_set,train_val_set   

In [5]:
'''train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)
print(len(train_set),len(test_set),len(val_set),len(train_val_set))
users_num=len(test_set)'''

'train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)\nprint(len(train_set),len(test_set),len(val_set),len(train_val_set))\nusers_num=len(test_set)'

In [6]:
'''users, click_seqs, pos_items,neg_items = zip(*train_set)
train_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}
users, click_seqs, pos_items,neg_items = zip(*test_set)
test_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}
users, click_seqs, pos_items,neg_items = zip(*val_set)
val_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}'''

"users, click_seqs, pos_items,neg_items = zip(*train_set)\ntrain_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}\nusers, click_seqs, pos_items,neg_items = zip(*test_set)\ntest_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}\nusers, click_seqs, pos_items,neg_items = zip(*val_set)\nval_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}"

In [7]:
'''with open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f, -1)
with open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f, -1)
with open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_val_data.pkl', 'wb') as f:
    pickle.dump(val_data, f, -1)'''

"with open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_train_data.pkl', 'wb') as f:\n    pickle.dump(train_data, f, -1)\nwith open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_test_data.pkl', 'wb') as f:\n    pickle.dump(test_data, f, -1)\nwith open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_val_data.pkl', 'wb') as f:\n    pickle.dump(val_data, f, -1)"

In [8]:
dataset_name='ml-100k'

with open('/root/autodl-nas/workspace/datasets/ml/ml-100k/'+dataset_name+'_train_data.pkl', 'rb') as f:
    train_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/ml/ml-100k/'+dataset_name+'_test_data.pkl', 'rb') as f:
    test_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/ml/ml-100k/'+dataset_name+'_val_data.pkl', 'rb') as f:
    val_data=pickle.load(f)

In [9]:
print(np.shape(train_data['neg_item']),np.shape(test_data['neg_item']),np.shape(val_data['neg_item']))

(52139, 5) (931, 1000) (931, 5)


In [10]:
def scaled_dot_product_attention(q, k, v, mask):
    """Attention Mechanism Function.
    Args:
        :param q: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param k: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param v: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param mask: A 3d/4d tensor with shape of (None, ..., seq_len, 1)
    :return:
    """
    mat_qk = tf.matmul(q, k, transpose_b=True)  # (None, seq_len, seq_len)
    # Scaled
    dk = tf.cast(k.shape[-1], dtype=tf.float32)
    scaled_att_logits = mat_qk / tf.sqrt(dk)

    paddings = tf.ones_like(scaled_att_logits) * (-2 ** 32 + 1)  # (None, seq_len, seq_len)
    if mask!=None:
        outputs = tf.where(tf.equal(mask, tf.zeros_like(mask)), paddings, scaled_att_logits)  # (None, seq_len, seq_len)
    else:
        outputs=scaled_att_logits
    # softmax
    outputs = tf.nn.softmax(logits=outputs)  # (None, seq_len, seq_len)
    outputs = tf.matmul(outputs, v)  # (None, seq_len, dim)

    return outputs


def split_heads(x, seq_len, num_heads, depth):
    """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    Args:
        :param x: A Tensor with shape of [batch_size, seq_len, num_heads * depth]
        :param seq_len: A scalar(int).
        :param num_heads: A scalar(int).
        :param depth: A scalar(int).
    :return: A tensor with shape of [batch_size, num_heads, seq_len, depth]
    """
    x = tf.reshape(x, (-1, seq_len, num_heads, depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

def normalize_adj_tensor(adj, seq_len):
    adj = adj + tf.expand_dims(tf.eye(seq_len), axis=0)
    rowsum = tf.reduce_sum(adj, axis=1)
    d_inv_sqrt = tf.pow(rowsum, -0.5)
    candidate_a = tf.zeros_like(d_inv_sqrt)
    d_inv_sqrt = tf.where(tf.math.is_inf(d_inv_sqrt), candidate_a, d_inv_sqrt)
    d_mat_inv_sqrt = tf.linalg.diag(d_inv_sqrt)
    norm_adg = tf.matmul(d_mat_inv_sqrt, adj)
    return norm_adg


def contrastive_label(seq_embed,att):
    s_item_emd,s_item_index=tf.math.top_k(att, k)
    s_item_index=tf.reshape(s_item_index,[-1,1])
    h_index=tf.sort(tf.tile(tf.reshape(tf.range(tf.shape(seq_embed)[0]),[-1,1]),[interest_num*k,1]),axis=0)
    index = tf.stack([h_index, s_item_index])
    index=tf.reshape(index,[-1,k,2])
    all_user_label=tf.gather_nd(seq_embed, index)
    all_user_label=tf.reshape(all_user_label,[-1,interest_num,k,hidden_size])
    all_user_label=tf.reduce_mean(all_user_label,axis=-2)
    return all_user_label

def density(x):
    sim=tf.matmul(x,tf.transpose(x,[0,2,1]))
    sim=sim*(1+(-1*2^10)*tf.linalg.band_part(tf.ones((len_seq,len_seq)),-1,0))
    sim,_=tf.math.top_k(sim, k=3)#取最大的3个
    return tf.math.reduce_mean(sim)

def new_density(x):
    x=tf.reshape(x,[-1,hidden_size])
    sim=tf.matmul(x,tf.transpose(x,[1,0]))
    sim=sim*(1+(-1*2^10)*tf.linalg.band_part(tf.ones_like(sim),-1,0))
    sim,_=tf.math.top_k(sim, k=10)#取最大的10个
    return tf.math.reduce_mean(sim)

In [11]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        """Multi Head Attention Mechanism.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads. If num_heads == 1, the layer is a single self-attention layer.
        :return:
        """
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        self.wq = tf.keras.layers.Dense(d_model, activation=None)
        self.wk = tf.keras.layers.Dense(d_model, activation=None)
        self.wv = tf.keras.layers.Dense(d_model, activation=None)

    def call(self, q, k, v, mask):
        q = self.wq(q)  # (None, seq_len, d_model)
        k = self.wk(k)  # (None, seq_len, d_model)
        v = self.wv(v)  # (None, seq_len, d_model)
        # split d_model into num_heads * depth
        seq_len, d_model = q.shape[1], q.shape[2]
        q = split_heads(q, seq_len, self.num_heads, q.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        k = split_heads(k, seq_len, self.num_heads, k.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        v = split_heads(v, seq_len, self.num_heads, v.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        # mask
        if mask!=None:
            mask = tf.tile(tf.expand_dims(mask, axis=1), [1, self.num_heads, 1, 1])  # (None, num_heads, seq_len, 1)
        # attention
        scaled_attention = scaled_dot_product_attention(q, k, v, mask)  # (None, num_heads, seq_len, d_model // num_heads)
        # reshape
        outputs = tf.reshape(tf.transpose(scaled_attention, [0, 2, 1, 3]), [-1, seq_len, d_model])  # (None, seq_len, d_model)
        return outputs


class FFN(tf.keras.layers.Layer):
    def __init__(self, hidden_unit, d_model):
        """Feed Forward Network.
        Args:
            :param hidden_unit: A scalar.
            :param d_model: A scalar.
        :return:
        """
        super(FFN, self).__init__()
        self.conv1 = tf.keras.layers.Conv1D(filters=hidden_unit, kernel_size=1, activation='relu', use_bias=True)
        self.conv2 = tf.keras.layers.Conv1D(filters=d_model, kernel_size=1, activation=None, use_bias=True)

    def call(self, inputs):
        x = self.conv1(inputs)
        output = self.conv2(x)
        return output


class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads=1, ffn_hidden_unit=128, dropout=0., layer_norm_eps=1e-6):
        """Encoder Layer.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads.
            :param ffn_hidden_unit: A scalar. Number of hidden unit in FFN
            :param dropout: A scalar. Number of dropout.
            :param layer_norm_eps: A scalar. Small float added to variance to avoid dividing by zero.
        :return:
        """
        super(TransformerEncoder, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = FFN(ffn_hidden_unit, d_model)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, inputs):
        x, mask= inputs
        # self-attention
        att_out = self.mha(x, x, x, mask)  # (None, seq_len, d_model)
        att_out = self.dropout1(att_out)
        # residual add
        out1 = self.layernorm1(x + att_out)  # (None, seq_len, d_model)
        # ffn
        ffn_out = self.ffn(out1)
        ffn_out = self.dropout2(ffn_out)
        # residual add
        out2 = self.layernorm2(out1 + ffn_out)  # (None, seq_len, d_model)
        return out2

class gcn_layer(tf.keras.layers.Layer):
    def __init__(self, hidden_size, num_layer):
        super(gcn_layer, self).__init__()
        #weights_size_list = [hidden_size,64,64,64]
        #all_weights = {}
        self.num_layer=num_layer
        self.conv_dense=tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
    
    def call(self,A, x):
    # gcn has three layers
        all_embeddings =[x]
        for k in range(self.num_layer):
            embeddings = tf.matmul(A, x)
            embeddings = self.conv_dense(embeddings)
            all_embeddings.append(embeddings)
        return all_embeddings

class light_gcn_layer(tf.keras.layers.Layer):
    def __init__(self, hidden_size, num_layer):
        super(gcn_layer, self).__init__()
        #weights_size_list = [hidden_size,64,64,64]
        #all_weights = {}
        self.num_layer=num_layer
        self.conv_dense=tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
    
    def call(self,A, x):
    # gcn has three layers
        all_embeddings =[x]
        for k in range(self.num_layer):
            embeddings = tf.matmul(A, x)
            #embeddings = self.conv_dense(embeddings)
            all_embeddings.append(embeddings)
        return all_embeddings    
    
class hierarchical_interest(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(hierarchical_interest, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        
        self.adj=tf.linalg.band_part(tf.ones((layers_num,layers_num)),-1,0)-1
        self.layer_infor={}
        for i in range(0,layers_num+1):
            self.layer_infor[str(i)]=tf.keras.layers.Dense(self.d_model, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
        
        self.conv_layer_dict={}
        for i in range(0,layers_num-1):
            self.conv_layer_dict['conv_layer_'+str(i)]=tf.nn.softmax(tf.Variable(initial_value=tf.random.truncated_normal([int((layers_num-1-i)*len_seq/(layers_num-1)),1,1,1],mean=0.5,stddev=1),name='w_'+str(i)),axis=0)

        self.conv_layer_dict['conv_layer_'+str(layers_num-1)]=tf.nn.softmax(tf.Variable(initial_value=tf.random.truncated_normal([1,1,1,1],mean=0.5,stddev=1),name='w_'+str(layers_num-1)),axis=0)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.layer_dif_weight=tf.nn.softmax(tf.Variable(initial_value=tf.ones((1,layers_num+1)),name='w_'+str(i)),axis=-1)
        
    def call(self, ori_all_embeddings):
        
        ori_all_embeddings[-1]=tf.nn.conv2d(tf.expand_dims(ori_all_embeddings[-1],-1),self.conv_layer_dict['conv_layer_0'],[1,1,1,1],'SAME')
        ori_all_embeddings[-1]=tf.squeeze(ori_all_embeddings[-1],-1)
        for i,element in enumerate(ori_all_embeddings):
            #print(i)
            if i==len(ori_all_embeddings)-1:
                break
            ori_all_embeddings[i]=tf.nn.conv2d(tf.expand_dims(element,-1),self.conv_layer_dict['conv_layer_'+str(layers_num-1-i)],[1,1,1,1],'SAME')
            ori_all_embeddings[i]=tf.squeeze(ori_all_embeddings[i],-1)
            ori_all_embeddings[i]=self.layernorm(ori_all_embeddings[i])
        
            
        all_embeddings_tensor=tf.stack(ori_all_embeddings,axis=1)#[b,Layers,l,h]
        renew_embeddings=[]
        for i in range(len_seq):
            layer_seq_embed=tf.gather(all_embeddings_tensor, indices=[i], axis=2)
            #print(tf.shape(layer_seq_embed))
            
            layer_seq_embed=tf.squeeze(layer_seq_embed,axis=2)
            #layer_seq_embed=tf.squeeze(layer_seq_embed,axis=2)
            '''
            adj_l = tf.tile(tf.expand_dims(layer_seq_embed, axis=2), [1, 1, layers_num+1, 1])
            adj_r = tf.tile(tf.expand_dims(layer_seq_embed, axis=1), [1, layers_num+1, 1, 1])
            adj =tf.nn.leaky_relu(tf.nn.sigmoid(tf.reduce_sum(adj_l * adj_r, axis=-1)),alpha=0.01)
            
            adj = normalize_adj_tensor(adj, layers_num+1)
            
            adj=tf.multiply(adj,tf.linalg.band_part(tf.ones((layers_num+1,layers_num+1)),-1,0)-1)
            
            renew_embeddings.append(self.layernorm(tf.matmul(adj, layer_seq_embed)))
            '''
            renew_embeddings.append(self.layernorm(layer_seq_embed))
        renew_embeddings_tensor=tf.stack(renew_embeddings,axis=2)
        all_embeddings=tf.unstack(renew_embeddings_tensor,axis=1)
        bef_sim=[]
        aft_sim=[]
        dif=[]
        for i,ele in enumerate(all_embeddings):
            #sim=tf.matmul(ele,tf.transpose(ele,[0,2,1]))
            #sim=sim*(1-tf.linalg.band_part(tf.ones((len_seq,len_seq)),-1,0))
            bef_sim.append(new_density(ele))
            
        for i,ele in enumerate(all_embeddings):
            all_embeddings[i]=self.layer_infor[str(i)](all_embeddings[i])
        for i,ele in enumerate(all_embeddings):
            #sim=tf.matmul(ele,tf.transpose(ele,[0,2,1]))
            #sim=sim*(1-tf.linalg.band_part(tf.ones((len_seq,len_seq)),-1,0))
            aft_sim.append(new_density(ele))
        for i,ele in enumerate(bef_sim):
            dif.append(tf.reduce_mean(tf.nn.sigmoid(bef_sim[i]-aft_sim[i])))
        dif=tf.reshape(tf.stack(dif,axis=0),[-1,1])#[l+1,1]
        all_embeddings.extend(ori_all_embeddings[0:-1])
        
        return all_embeddings,tf.math.reduce_sum(tf.matmul(self.layer_dif_weight,dif))

class CapsuleNetwork(tf.keras.layers.Layer):
    def __init__(self, embed_dim, seq_len, bilinear_type=0, num_interest=4, stop_grad=True):
        super(CapsuleNetwork, self).__init__()
        self.bilinear_type = bilinear_type
        self.seq_len = seq_len
        self.num_interest = num_interest
        self.embed_dim = embed_dim
        self.bi_dense=tf.keras.layers.Dense(hidden_size * self.num_interest, activation=None,use_bias=False)
        self.lstm=tf.keras.layers.LSTM(self.embed_dim*2, unit_forget_bias=1.0,return_sequences=True)
        self.stop_grad = stop_grad

    def build(self, input_shape):
        if self.bilinear_type >= 2:
            self.w = self.add_weight(
                shape=[1, self.seq_len, self.num_interest * self.embed_dim, self.embed_dim],
                initializer='random_normal',
                name='weights'
            )

    def call(self, hist_emb, mask):
        if self.bilinear_type == 0:
            hist_emb_hat = tf.tile(hist_emb, [1, 1, self.num_interest])  # (None, seq_len, num_inter * embed_dim)
        elif self.bilinear_type == 1:
            #outputs = self.lstm(hist_emb)
            #output = tf.concat(outputs, axis=0)
            #output = tf.reshape(output, (-1, self.seq_len, self.embed_dim*2))
            #output = tf.layers.dense(output, self.embed_dim*self.num_interest, activation=None, use_bias=False)
            hist_emb_hat = self.bi_dense(hist_emb)
        else:
            u = tf.expand_dims(hist_emb, axis=2)  # (None, seq_len, 1, embed_dim)
            hist_emb_hat = tf.reduce_sum(self.w * u, axis=3)  # (None, seq_len, num_inter * embed_dim)
        hist_emb_hat = tf.reshape(hist_emb_hat, [-1, self.seq_len, self.num_interest, self.embed_dim])
        hist_emb_hat = tf.transpose(hist_emb_hat, [0, 2, 1, 3])  # (None, num_inter, seq_len, embed_dim)
        hist_emb_hat = tf.reshape(hist_emb_hat, [-1, self.num_interest, self.seq_len, self.embed_dim])
        if self.stop_grad:
            hist_emb_iter = tf.stop_gradient(hist_emb_hat)
        else:
            hist_emb_iter = hist_emb_hat  # (None, num_inter, seq_len, embed_dim)

        if self.bilinear_type > 0:
            '''self.capsule_weight = self.add_weight(
                shape=[tf.shape(hist_emb_hat)[0], self.num_interest, self.seq_len],
                initializer=tf.zeros_initializer())'''
            self.capsule_weight = tf.random.truncated_normal(
                shape=[tf.shape(hist_emb_hat)[0], self.num_interest, self.seq_len],
                stddev=1.0)
            
        else:
            self.capsule_weight = tf.random.truncated_normal(
                shape=[tf.shape(hist_emb_hat)[0], self.num_interest, self.seq_len],
                stddev=1.0)
        tf.stop_gradient(self.capsule_weight)

        for i in range(3):
            att_mask = tf.tile(tf.expand_dims(mask, axis=1), [1, self.num_interest, 1])  # (None, num_inter, seq_len)
            paddings = tf.zeros_like(att_mask)

            capsule_softmax_weight = tf.nn.softmax(self.capsule_weight, axis=1)  # (None, num_inter, seq_len)
            capsule_softmax_weight = tf.where(tf.equal(att_mask, 0), paddings, capsule_softmax_weight)
            capsule_softmax_weight = tf.expand_dims(capsule_softmax_weight, 2)  # (None, num_inter, 1, seq_len)

            if i < 2:
                interest_capsule = tf.matmul(capsule_softmax_weight, hist_emb_iter)  # (None, num_inter, 1, embed_dim)
                cap_norm = tf.reduce_sum(tf.square(interest_capsule), -1, keepdims=True)
                scalar_factor = cap_norm / (1 + cap_norm) / tf.sqrt(cap_norm + 1e-9)
                interest_capsule = scalar_factor * interest_capsule

                delta_weight = tf.matmul(hist_emb_iter, tf.transpose(interest_capsule, [0, 1, 3, 2]))
                delta_weight = tf.reshape(delta_weight, [-1, self.num_interest, self.seq_len])
                self.capsule_weight = self.capsule_weight + delta_weight
            else:
                interest_capsule = tf.matmul(capsule_softmax_weight, hist_emb_hat)
                cap_norm = tf.reduce_sum(tf.square(interest_capsule), -1, True)
                scalar_factor = cap_norm / (1 + cap_norm) / tf.sqrt(cap_norm + 1e-9)
                interest_capsule = scalar_factor * interest_capsule

        interest_capsule = tf.reshape(interest_capsule, [-1, self.num_interest, self.embed_dim])
        return interest_capsule

In [12]:
class mgnm(tf.keras.models.Model):
    def __init__(self):
        super(mgnm, self).__init__()
        blocks=1
        embed_reg=0.
        layer_norm_eps=1e-6
        num_heads=1
        use_l2norm=True
        
        self.len_seq=len_seq
        self.item_embedding = tf.keras.layers.Embedding(items_num+1,hidden_size,input_length=self.len_seq,
                                                       embeddings_initializer='random_normal',
                                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.pos_embedding = tf.keras.layers.Embedding(len_seq,hidden_size,input_length=1,
                                                       embeddings_initializer='random_normal',
                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.dropout = tf.keras.layers.Dropout(1-keep_rate)
        self.transformer_encoder_layer={}
        for i in range(0,layers_num+1):
            self.transformer_encoder_layer['encoder_layer_'+str(i)] = [TransformerEncoder(hidden_size, num_heads, 128,
                                                 (1-keep_rate), layer_norm_eps) for _ in range(blocks)]
        self.hiera=hierarchical_interest(hidden_size,0)
        self.encoder_layer_dict={}
        for i in range(0,2*(layers_num+1)):
            self.encoder_layer_dict['encoder_layer_'+str(i)]=CapsuleNetwork(hidden_size, len_seq, 1, interest_num, False)
        
        '''
        self.multi_interest_weight={}
        for i in range(0,2*(layers_num+1)):
            self.multi_interest_weight['multi_interest_weight_'+str(i)]=tf.Variable(initial_value=tf.random.truncated_normal([interest_num,len_seq],mean=0.5,stddev=1),name='w_'+str(i-1))
        '''
        self.multi_interest_dict={}
        for i in range(0,2*(layers_num+1)):
            self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w1']=tf.keras.layers.Dense(hidden_size * 4, activation='tanh')
            self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w2']=tf.keras.layers.Dense(hidden_size *interest_num, activation=None)
        
        self.gru_encoder={}
        for i in range(0,2*(layers_num+1)):
            self.gru_encoder['gru_encoder'+str(i)] = tf.keras.layers.GRU(hidden_size)
        
        self.gcn=gcn_layer(hidden_size, layers_num)
        self.score_weight=tf.nn.sigmoid(tf.Variable(initial_value=tf.ones((2*layers_num+1,1)),name='w_'+str(i-1)))
        # norm
        self.use_l2norm = use_l2norm
        
    
    def call(self, inputs):
        seq_embed=self.item_embedding(inputs['click_seq'])
        mask_1=tf.expand_dims(tf.cast(tf.sequence_mask(tf.range(1,tf.shape(seq_embed)[0]+1),len_seq), dtype=tf.float32), axis=-1)
        mask_2 = tf.expand_dims(tf.cast(tf.not_equal(inputs['click_seq'], 0), dtype=tf.float32), axis=-1)  # (None, seq_len, 1)
        mask=tf.multiply(mask_1,mask_2)
        
        adj_l = tf.tile(tf.expand_dims(seq_embed, axis=2), [1, 1, len_seq, 1])
        adj_r = tf.tile(tf.expand_dims(seq_embed, axis=1), [1, len_seq, 1, 1])
        adj = tf.nn.sigmoid(tf.reduce_sum(adj_l * adj_r, axis=-1))
        self.adj_l1 = tf.norm(adj, ord=1)
        
        if 1:
            adj = normalize_adj_tensor(adj, len_seq) 
        
        all_embeddings=self.gcn(adj,seq_embed)
        
        all_embeddings,dif=self.hiera(all_embeddings)
        
        pos_encoding = tf.expand_dims(self.pos_embedding(tf.range(self.len_seq)), axis=0)  # (1, seq_len, embed_dim)
        att_outputs_dict={}
        for i,element in enumerate(all_embeddings):
            #print(i,'/',layers_num)
            #att_outputs_dict['embed_'+str(i)]=tf.math.reduce_mean(element,axis=1,keepdims=True)
            att_outputs_dict['embed_'+str(i)]=element+pos_encoding
            
            att_outputs_dict['embed_'+str(i)] = self.dropout(att_outputs_dict['embed_'+str(i)])
        

        multi_interest_weight={}
        for i in range(0,2*layers_num+1):
            att=self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w1'](att_outputs_dict['embed_'+str(i)])#[b,len_seq,h]
            att=self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w2'](att)#[b,len_seq,interest_num*h]
            #att=tf.transpose(att, [0, 2, 1])#[b,num_heads,len_seq]
            #att=tf.nn.softmax(att)
            
            #att_outputs_dict['embed_'+str(i)]=tf.expand_dims(att_outputs_dict['embed_'+str(i)],axis=1)#[b,1,l,h]
            
            
            
            #multi_interest_weight['multi_interest_weight_'+str(i)]=tf.expand_dims(self.multi_interest_weight['multi_interest_weight_'+str(i)],axis=0)#[1,k,l]
            #multi_interest_weight['multi_interest_weight_'+str(i)]=tf.expand_dims(att,axis=-1)#[b,k,l,1]

            #att_outputs_dict['embed_'+str(i)]=tf.multiply(att_outputs_dict['embed_'+str(i)],multi_interest_weight['multi_interest_weight_'+str(i)])#[b,k,l,h]
            
            att_outputs_dict['embed_'+str(i)]=tf.reshape(att,[-1,interest_num,len_seq,hidden_size])
            
            att_outputs_dict['embed_'+str(i)]=tf.unstack(att_outputs_dict['embed_'+str(i)],axis=1)#k*[b,l,h]
        
        self.layer_interest_dict={}
        for i in range(0,2*layers_num+1):
            self.layer_interest_dict['layer_'+str(i)+'_interest']=[]
            if type(att_outputs_dict['embed_'+str(i)])!=list:
                        att_outputs_dict['embed_'+str(i)]=[att_outputs_dict['embed_'+str(i)]]
            for j in range(interest_num):
                '''
                if i>3:
                    k=i-4
                else:
                    k=i
                        
                for block in self.transformer_encoder_layer['encoder_layer_'+str(k)]:
                    
                    att_outputs = block([(att_outputs_dict['embed_'+str(i)][j]),mask])  # (None, seq_len, embed_dim)
                    interest = tf.slice(att_outputs, begin=[0, self.len_seq-1, 0], size=[-1, 1, -1])
                '''    
                interest=self.gru_encoder['gru_encoder'+str(i)](att_outputs_dict['embed_'+str(i)][j])

                self.layer_interest_dict['layer_'+str(i)+'_interest'].append(interest)#[b,h]
                
            self.layer_interest_dict['layer_'+str(i)+'_interest']=tf.stack(self.layer_interest_dict['layer_'+str(i)+'_interest'],1)#[b,k,h]
            self.layer_interest_dict['layer_'+str(i)+'_interest']=tf.reshape(self.layer_interest_dict['layer_'+str(i)+'_interest'],
                                                                                 [-1,1,interest_num,hidden_size])
            

        mixed_interest=tf.concat([self.layer_interest_dict['layer_'+str(i)+'_interest'] for i in range(0,2*layers_num+1)],axis=1)#[b,l,n,h]
        '''
        pos_emb=self.item_embedding(tf.reshape(inputs['pos_item'], [-1, ]))#[b,h]
        neg_emb=self.item_embedding(inputs['neg_item'])#[b,neg_num,h]
        
        pos_score=tf.zeros([tf.shape(pos_emb)[0],1])
        neg_score=tf.zeros([tf.shape(neg_emb)[0],tf.shape(neg_emb)[1]])

        for i in range(0,layers_num+1):
            pos_score+=tf.reduce_max(tf.reduce_sum(tf.multiply(tf.squeeze(self.layer_interest_dict['layer_'+str(i)+'_interest'],axis=1),tf.expand_dims(pos_emb,axis=1))),keepdims=True)
        
        for i in range(0,layers_num+1):
            neg_score+=tf.reduce_max(tf.matmul(neg_emb,tf.transpose(tf.squeeze(self.layer_interest_dict['layer_'+str(i)+'_interest'],axis=1),[0,2,1])))
        print(tf.shape(pos_score),tf.shape(neg_score))
        '''
        
        #print(tf.shape(mixed_interest))
        pos_emb=self.item_embedding(tf.reshape(inputs['pos_item'], [-1, ]))#[b,h]
        neg_emb=self.item_embedding(inputs['neg_item'])#[b,neg_num,h]
        
        if self.use_l2norm:
            pos_emb = tf.math.l2_normalize(pos_emb, axis=-1)
            neg_emb = tf.math.l2_normalize(neg_emb, axis=-1)
            mixed_interest = tf.math.l2_normalize(mixed_interest, axis=-1)
        expanded_target_item=tf.expand_dims(tf.expand_dims(pos_emb,axis=1),axis=1)#[b,1,1,h]
        expaned_neg_emb=tf.expand_dims(neg_emb,axis=1)#[b,1,neg_num,h]

        
        alpha=tf.reduce_max(tf.matmul(expanded_target_item,tf.transpose(mixed_interest,[0,1,3,2])),axis=-1)#[b,l,1]
        pos_score=tf.squeeze(tf.matmul(tf.transpose(alpha,[0,2,1]),self.score_weight),axis=-1)#[b,1]
            
        alpha=tf.reduce_max(tf.matmul(expaned_neg_emb,tf.transpose(mixed_interest,[0,1,3,2])),axis=-1)#[b,l,neg_num]
        neg_score=tf.squeeze(tf.matmul(tf.transpose(alpha,[0,2,1]),self.score_weight),axis=-1)#[b,neg_num]
        
        '''
        alpha=tf.matmul(expanded_target_item,tf.transpose(mixed_interest,[0,1,3,2]))#[b,l,1,n]
        alpha=tf.nn.softmax(alpha)
        res = tf.matmul(alpha,mixed_interest)#[b,l,1,h]
        pos_score=tf.reduce_sum(tf.reduce_sum(tf.multiply(res,expanded_target_item),axis=-1),axis=-2)#[b,1]
            
        
        
        alpha=tf.matmul(expaned_neg_emb,tf.transpose(mixed_interest,[0,1,3,2]))#[b,l,neg_num,n]
        alpha=tf.nn.softmax(alpha)
        res = tf.matmul(alpha,mixed_interest)#[b,l,n,h]
        neg_score=tf.reduce_sum(tf.reduce_sum(tf.multiply(res,expaned_neg_emb),axis=-1),axis=-2)#[b,neg_num]
        '''
        logits = tf.nn.softmax(tf.concat([pos_score, neg_score], axis=-1))
        #pos_score = tf.tile(pos_score, [1, neg_score.shape[1]])
        
        loss = tf.reduce_mean(- tf.math.log(logits)[:,0])+dif
        self.add_loss(loss)
        
        return logits
    
    def summary(self):
        inputs = {
            'click_seq': tf.keras.layers.Input(shape=(self.len_seq,), dtype=tf.int32),
            'pos_item': tf.keras.layers.Input(shape=(), dtype=tf.int32),
            'neg_item': tf.keras.layers.Input(shape=(1,), dtype=tf.int32)  # suppose neg_num=1
        }
        tf.keras.models.Model(inputs=inputs, outputs=self.call(inputs)).summary()

In [13]:
def hr(rank, k):
    """Hit Rate.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: hit rate.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1
    return res / len(rank)


def mrr(rank, k):
    """Mean Reciprocal Rank.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: mrr.
    """
    mrr = 0.0
    for r in rank:
        if r < k:
            mrr += 1 / (r + 1)
    return mrr / len(rank)


def ndcg(rank, k):
    """Normalized Discounted Cumulative Gain.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: ndcg.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1 / np.log2(r + 2)
    return res / len(rank)


def eval_rank(pred_y, metric_names, k=10):
    """Evaluate
        Args:
            :param pred_y: A ndarray.
            :param metric_names: A list like ['hr'].
            :param k: A scalar(int).
        :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    rank = pred_y.argsort().argsort()[:, 0]
    res_dict = {}
    for name in metric_names:
        if name == 'hr':
            res = hr(rank, k)
        elif name == 'ndcg':
            res = ndcg(rank, k)
        elif name == 'mrr':
            res = mrr(rank, k)
        else:
            break
        res_dict[name] = res
    return res_dict

def eval_pos_neg(model, test_data, metric_names, k=10, batch_size=None):
    """Evaluate the performance of Top-k recommendation algorithm.
    Note: Test data must contain some negative samples(>= k) and one positive samples.
    Args:
        :param model: A model built-by tensorflow.
        :param test_data: A dict.
        :param metric_names: A list like ['hr'].
        :param k: A scalar(int).
        :param batch_size: A scalar(int).
    :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    pred_y = - model.predict(test_data, batch_size)
    print(np.shape(pred_y))
    return eval_rank(pred_y, metric_names, k)

In [14]:
'''callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
from time import time
strategy = tf.distribute.MirroredStrategy()
batch_size=batch_size*strategy.num_replicas_in_sync
with strategy.scope():
    model = mgnm()
    model.summary()
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(optimizer=optimizer)

model.fit(
            x=train_data,
            epochs=100,
            batch_size=batch_size,
        validation_data=val_data,
    callbacks=[callback])
t2=time()
eval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)
print('Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (time() - t2,eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))'''

"callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)\nfrom time import time\nstrategy = tf.distribute.MirroredStrategy()\nbatch_size=batch_size*strategy.num_replicas_in_sync\nwith strategy.scope():\n    model = mgnm()\n    model.summary()\n    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)\n    model.compile(optimizer=optimizer)\n\nmodel.fit(\n            x=train_data,\n            epochs=100,\n            batch_size=batch_size,\n        validation_data=val_data,\n    callbacks=[callback])\nt2=time()\neval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)\nprint('Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'\n              % (time() - t2,eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))"

In [15]:
from time import time
strategy = tf.distribute.MirroredStrategy()
batch_size=batch_size*strategy.num_replicas_in_sync
with strategy.scope():
    model = mgnm()
    model.summary()
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(optimizer=optimizer)
best_eval_dict={'hr':0.,'mrr':0.,'ndcg':0.}
eval_patience={'hr':0.,'mrr':0.,'ndcg':0.}

for epoch in range(1, epoch_num + 1):
    t1 = time()
    model.fit(
            x=train_data,
            epochs=1,
            batch_size=batch_size,
        validation_data=val_data)
    t2 = time()
    eval_dict = eval_pos_neg(model, val_data, ['hr', 'mrr', 'ndcg'], 1, batch_size)
    print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))
    for i in eval_dict.keys():
        if eval_dict[i]>=best_eval_dict[i]:
            best_eval_dict[i]=eval_dict[i]
        else:
            print(i,'drop 1 time')
            eval_patience[i]+=1
    if (np.array(list(eval_patience.values()))>=3).any():
        print('Trigger early stopping')
        break
eval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)
print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))   

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
tf.reshape_14 (TFOpLambda)      (None,)              0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 10)]         0                                            
____________________________________________________

In [16]:
#final version

#L=1 K=1 independent GRU ausloss with embedding_weight HR = 0.1289, MRR = 0.0411, NDCG = 0.0612;
#L=1 K=2 independent GRU ausloss with embedding_weight HR = 0.1171, MRR = 0.0354, NDCG = 0.0541
#L=1 K=3 independent GRU ausloss with embedding_weight HR = 0.1085, MRR = 0.0293, NDCG = 0.0474
#L=1 K=4 independent GRU ausloss with embedding_weight HR = 0.1074, MRR = 0.0368, NDCG = 0.0528
#L=1 K=5 independent GRU ausloss with embedding_weight HR = 0.1063, MRR = 0.0311, NDCG = 0.0484

#L=2 K=1 independent GRU ausloss with embedding_weight HR = 0.1321, MRR = 0.0423, NDCG = 0.0629; HR = 0.1332, MRR = 0.0491, NDCG = 0.0685
#L=2 K=2 independent GRU ausloss with embedding_weight HR = 0.1343, MRR = 0.0401, NDCG = 0.0615; HR = 0.1214, MRR = 0.0428, NDCG = 0.0608
#L=2 K=3 independent GRU ausloss with embedding_weight HR = 0.1257, MRR = 0.0355, NDCG = 0.0563; HR = 0.1396, MRR = 0.0429, NDCG = 0.0648
#L=2 K=4 independent GRU ausloss with embedding_weight HR = 0.1257, MRR = 0.0416, NDCG = 0.0609; HR = 0.1235, MRR = 0.0389, NDCG = 0.0583
#L=2 K=5 independent GRU ausloss with embedding_weight HR = 0.1289, MRR = 0.0404, NDCG = 0.0607; HR = 0.1160, MRR = 0.0346, NDCG = 0.0533

#L=3 K=1 independent GRU ausloss with embedding_weight HR = 0.1450, MRR = 0.0539, NDCG = 0.0747; HR = 0.1525, MRR = 0.0543, NDCG = 0.0769
#L=3 K=2 independent GRU ausloss with embedding_weight HR = 0.1407, MRR = 0.0433, NDCG = 0.0654; HR = 0.1386, MRR = 0.0405, NDCG = 0.0629
#L=3 K=3 independent GRU ausloss with embedding_weight HR = 0.1364, MRR = 0.0519, NDCG = 0.0713; HR = 0.1386, MRR = 0.0449, NDCG = 0.0664
#L=3 K=4 independent GRU ausloss with embedding_weight HR = 0.1364, MRR = 0.0467, NDCG = 0.0674; HR = 0.1278, MRR = 0.0436, NDCG = 0.0630
#L=3 K=5 independent GRU ausloss with embedding_weight HR = 0.1439, MRR = 0.0461, NDCG = 0.0686; HR = 0.1364, MRR = 0.0421, NDCG = 0.0637

#L=4 K=1 independent GRU ausloss with embedding_weight HR = 0.1396, MRR = 0.0569, NDCG = 0.0758; HR = 0.1504, MRR = 0.0529, NDCG = 0.0754
#L=4 K=2 independent GRU ausloss with embedding_weight HR = 0.1375, MRR = 0.0416, NDCG = 0.0637; HR = 0.1439, MRR = 0.0488, NDCG = 0.0706
#L=4 K=3 independent GRU ausloss with embedding_weight HR = 0.1536, MRR = 0.0463, NDCG = 0.0708; HR = 0.1429, MRR = 0.0493, NDCG = 0.0708
#L=4 K=4 independent GRU ausloss with embedding_weight HR = 0.1461, MRR = 0.0482, NDCG = 0.0707; HR = 0.1450, MRR = 0.0490, NDCG = 0.0712
#L=4 K=5 independent GRU ausloss with embedding_weight HR = 0.1418, MRR = 0.0493, NDCG = 0.0707; HR = 0.1439, MRR = 0.0511, NDCG = 0.0724


#L=5 K=4 independent GRU ausloss with embedding_weight HR = 0.1321, MRR = 0.0415, NDCG = 0.0625

In [17]:
#L=5 K=1 independent GRU ausloss with embedding_weight Iteration 16 Fit [1.7 s], Evaluate [19.2 s]: HR = 0.1257, MRR = 0.0366, NDCG = 0.0569
#L=5 K=2 independent GRU ausloss with embedding_weight Iteration 14 Fit [2.1 s], Evaluate [28.3 s]: HR = 0.1289, MRR = 0.0431, NDCG = 0.0629

#L=4 K=1 independent GRU ausloss with embedding_weight Iteration 13 Fit [1.5 s], Evaluate [15.2 s]: HR = 0.1418, MRR = 0.0502, NDCG = 0.0714
#L=4 K=2 independent GRU ausloss with embedding_weight Iteration 15 Fit [1.9 s], Evaluate [23.4 s]: HR = 0.1375, MRR = 0.0466, NDCG = 0.0673

#L=3 K=1 independent GRU ausloss Iteration 11 Fit [1.7 s], Evaluate [7.1 s]: HR = 0.1375, MRR = 0.0443, NDCG = 0.0656
#L=3 K=1 independent GRU ausloss with embedding_weight Iteration 10 Fit [1.9 s], Evaluate [7.4 s]: HR = 0.1192, MRR = 0.0445, NDCG = 0.0617
#L=3 K=2 independent GRU ausloss with embedding_weight Iteration 9 Fit [2.2 s], Evaluate [10.9 s]: HR = 0.1343, MRR = 0.0459, NDCG = 0.0663
#L=3 K=3 independent GRU ausloss with embedding_weight Iteration 13 Fit [2.9 s], Evaluate [13.3 s]: HR = 0.1482, MRR = 0.0417, NDCG = 0.0661
#L=3 K=4 independent GRU ausloss with embedding_weight Iteration 12 Fit [3.4 s], Evaluate [16.5 s]: HR = 0.1461, MRR = 0.0419, NDCG = 0.0657
#L=3 K=5 independent GRU ausloss with embedding_weight Iteration 10 Fit [3.5 s], Evaluate [19.8 s]: HR = 0.1278, MRR = 0.0439, NDCG = 0.0631

#L=2 K=1 independent GRU ausloss Iteration 11 Fit [1.7 s], Evaluate [7.1 s]: HR = 0.1375, MRR = 0.0443, NDCG = 0.0656
#L=2 K=1 independent GRU ausloss with embedding_weight Iteration 16 Fit [1.2 s], Evaluate [12.7 s]: HR = 0.1504, MRR = 0.0478, NDCG = 0.0713
#L=2 K=2 independent GRU ausloss with embedding_weight Iteration 21 Fit [1.4 s], Evaluate [17.7 s]: HR = 0.1622, MRR = 0.0499, NDCG = 0.0756
#L=2 K=3 independent GRU ausloss with embedding_weight Iteration 20 Fit [1.6 s], Evaluate [22.5 s]: HR = 0.1214, MRR = 0.0444, NDCG = 0.0619
#L=2 K=4 independent GRU ausloss with embedding_weight Iteration 15 Fit [1.8 s], Evaluate [27.8 s]: HR = 0.1321, MRR = 0.0432, NDCG = 0.0636
#L=2 K=5 independent GRU ausloss with embedding_weight Iteration 12 Fit [2.0 s], Evaluate [33.6 s]: HR = 0.1224, MRR = 0.0427, NDCG = 0.0611




#L=3 K=1 independent Trans ausloss 
#L=3 K=1 independent Trans ausloss with embedding_weight Iteration 12 Fit [2.7 s], Evaluate [5.9 s]: HR = 0.1192, MRR = 0.0424, NDCG = 0.0601
#L=3 K=2 independent Trans ausloss with embedding_weight Iteration 12 Fit [3.6 s], Evaluate [9.1 s]: HR = 0.1289, MRR = 0.0387, NDCG = 0.0593
#L=3 K=3 independent Trans ausloss with embedding_weight Iteration 12 Fit [4.4 s], Evaluate [10.3 s]: HR = 0.1418, MRR = 0.0436, NDCG = 0.0664
#L=3 K=4 independent Trans ausloss with embedding_weight Iteration 12 Fit [5.0 s], Evaluate [12.7 s]: HR = 0.1182, MRR = 0.0390, NDCG = 0.0574
#L=3 K=5 independent Trans ausloss with embedding_weight Iteration 12 Fit [5.8 s], Evaluate [15.9 s]: HR = 0.1386, MRR = 0.0420, NDCG = 0.0642
#L=3 K=6 independent Trans ausloss with embedding_weight Iteration 10 Fit [7.1 s], Evaluate [17.3 s]: HR = 0.1321, MRR = 0.0416, NDCG = 0.0624

#L=2 K=1 independent Trans ausloss 
#L=2 K=1 independent Trans ausloss with embedding_weight Iteration 11 Fit [2.2 s], Evaluate [5.3 s]: HR = 0.1364, MRR = 0.0434, NDCG = 0.0646
#L=2 K=2 independent Trans ausloss with embedding_weight Iteration 12 Fit [2.9 s], Evaluate [7.7 s]: HR = 0.1106, MRR = 0.0419, NDCG = 0.0575
#L=2 K=3 independent Trans ausloss with embedding_weight Iteration 14 Fit [3.3 s], Evaluate [9.0 s]: HR = 0.1353, MRR = 0.0453, NDCG = 0.0661
#L=2 K=4 independent Trans ausloss with embedding_weight Iteration 13 Fit [3.9 s], Evaluate [10.3 s]: HR = 0.1160, MRR = 0.0395, NDCG = 0.0571
#L=2 K=5 independent Trans ausloss with embedding_weight Iteration 11 Fit [4.3 s], Evaluate [12.6 s]: HR = 0.1139, MRR = 0.0333, NDCG = 0.0518

#L=4 K=1 independent Trans ausloss 
#L=4 K=1 independent Trans ausloss with embedding_weight Iteration 11 Fit [3.0 s], Evaluate [6.7 s]: HR = 0.1407, MRR = 0.0482, NDCG = 0.0697
#L=4 K=2 independent Trans ausloss with embedding_weight Iteration 11 Fit [4.2 s], Evaluate [9.4 s]: HR = 0.1300, MRR = 0.0435, NDCG = 0.0633
#L=4 K=3 independent Trans ausloss with embedding_weight Iteration 10 Fit [5.0 s], Evaluate [12.5 s]: HR = 0.1203, MRR = 0.0413, NDCG = 0.0595
#L=4 K=4 independent Trans ausloss with embedding_weight Iteration 12 Fit [6.1 s], Evaluate [15.4 s]: HR = 0.1214, MRR = 0.0364, NDCG = 0.0560
#L=4 K=5 independent Trans ausloss with embedding_weight Iteration 11 Fit [7.0 s], Evaluate [20.1 s]: HR = 0.1160, MRR = 0.0408, NDCG = 0.0581

#L=5 K=1 independent Trans ausloss 
#L=5 K=1 independent Trans ausloss with embedding_weight Iteration 13 Fit [3.5 s], Evaluate [8.2 s]: HR = 0.1235, MRR = 0.0479, NDCG = 0.0653
#L=5 K=2 independent Trans ausloss with embedding_weight Iteration 12 Fit [4.8 s], Evaluate [11.7 s]: HR = 0.1557, MRR = 0.0513, NDCG = 0.0753
#L=5 K=3 independent Trans ausloss with embedding_weight Iteration 11 Fit [6.3 s], Evaluate [15.5 s]: HR = 0.1386, MRR = 0.0418, NDCG = 0.0640
#L=5 K=4 independent Trans ausloss with embedding_weight Iteration 11 Fit [7.3 s], Evaluate [19.1 s]: HR = 0.1149, MRR = 0.0337, NDCG = 0.0524
#L=5 K=5 independent Trans ausloss with embedding_weight Iteration 11 Fit [8.6 s], Evaluate [21.7 s]: HR = 0.1343, MRR = 0.0437, NDCG = 0.0646

#L=6 K=1 independent Trans ausloss 
#L=6 K=1 independent Trans ausloss with embedding_weight Iteration 12 Fit [4.1 s], Evaluate [8.5 s]: HR = 0.1085, MRR = 0.0360, NDCG = 0.0526
#L=6 K=2 independent Trans ausloss with embedding_weight Iteration 12 Fit [5.8 s], Evaluate [12.4 s]: HR = 0.1353, MRR = 0.0444, NDCG = 0.0653
#L=6 K=3 independent Trans ausloss with embedding_weight Iteration 11 Fit [6.8 s], Evaluate [16.4 s]: HR = 0.1149, MRR = 0.0358, NDCG = 0.0538

In [18]:
#maxlen=1000
#len_seq=100
#batch_size=512
#epoch_num=100
#hidden_size=64
#keep_rate=0.9
#layers_num=10
#num_heads=1
#interest_num=3
#kernel_size=4
#transformerencoder=0
#neg_num=4
#test_neg_num=100
#Iteration 10 Fit [58.3 s], Evaluate [0.3 s]: HR = 0.2941, MRR = 0.0970, NDCG = 0.1424

In [19]:
'''def train_model(model,ds_train,ds_valid,epoches):

    for epoch in tf.range(1,epoches+1):
        model.reset_metrics()
        
        # 在后期降低学习率
        if epoch == 5000:
            model.optimizer.lr.assign(model.optimizer.lr/2.0)
            tf.print("Lowering optimizer Learning Rate...\n\n")
        
        for x, y in ds_train:
            train_result = model.train_on_batch(x, y)

        for x, y in ds_valid:
            valid_result = model.test_on_batch(x, y,reset_metrics=False)
            
        if epoch%100 ==0:
            tf.print("epoch = ",epoch)
            print("train:",dict(zip(model.metrics_names,train_result)))
            print("valid:",dict(zip(model.metrics_names,valid_result)))
            print("")'''

'def train_model(model,ds_train,ds_valid,epoches):\n\n    for epoch in tf.range(1,epoches+1):\n        model.reset_metrics()\n        \n        # 在后期降低学习率\n        if epoch == 5000:\n            model.optimizer.lr.assign(model.optimizer.lr/2.0)\n            tf.print("Lowering optimizer Learning Rate...\n\n")\n        \n        for x, y in ds_train:\n            train_result = model.train_on_batch(x, y)\n\n        for x, y in ds_valid:\n            valid_result = model.test_on_batch(x, y,reset_metrics=False)\n            \n        if epoch%100 ==0:\n            tf.print("epoch = ",epoch)\n            print("train:",dict(zip(model.metrics_names,train_result)))\n            print("valid:",dict(zip(model.metrics_names,valid_result)))\n            print("")'

In [20]:
'''train_model(model,ds_train,ds_test,800)'''

'train_model(model,ds_train,ds_test,800)'

In [21]:
import tensorflow as tf
params = tf.constant([[[0, 1.0, 2.0],
                      [10.0, 11.0, 12.0],
                      [20.0, 21.0, 22.0],
                      [30.0, 31.0, 32.0]],
                      
                      [[0, 1.0, 2.0],
                      [10.0, 11.0, 12.0],
                      [20.0, 21.0, 22.0],
                      [30.0, 31.0, 32.0]]])
tf.gather(params, indices=[1], axis=1).numpy()

array([[[10., 11., 12.]],

       [[10., 11., 12.]]], dtype=float32)

In [22]:
import tensorflow as tf
import numpy as np
a=tf.constant(np.random.random((5,3,4)))
c=tf.unstack(a,axis=0)
for i in c:
    print(i)
    print(np.corrcoef(i,i)[0:5,0:5])

tf.Tensor(
[[0.39526348 0.01243695 0.43185352 0.80750525]
 [0.24899643 0.4769238  0.61475592 0.20803588]
 [0.3154374  0.04143804 0.11443536 0.09920483]], shape=(3, 4), dtype=float64)
[[ 1.         -0.53259367  0.1701367   1.         -0.53259367]
 [-0.53259367  1.         -0.45845229 -0.53259367  1.        ]
 [ 0.1701367  -0.45845229  1.          0.1701367  -0.45845229]
 [ 1.         -0.53259367  0.1701367   1.         -0.53259367]
 [-0.53259367  1.         -0.45845229 -0.53259367  1.        ]]
tf.Tensor(
[[0.04637704 0.86176017 0.61235747 0.11658125]
 [0.25098025 0.78633697 0.08936918 0.9350005 ]
 [0.58965874 0.25158053 0.01969382 0.09827373]], shape=(3, 4), dtype=float64)
[[ 1.          0.01913527 -0.41985733  1.          0.01913527]
 [ 0.01913527  1.         -0.17758429  0.01913527  1.        ]
 [-0.41985733 -0.17758429  1.         -0.41985733 -0.17758429]
 [ 1.          0.01913527 -0.41985733  1.          0.01913527]
 [ 0.01913527  1.         -0.17758429  0.01913527  1.        ]]
tf

In [23]:
np.corrcoef([[0.50142582 0.02288794 0.9867883  0.06646458]
 [0.71259711 0.73245141 0.96514089 0.75292946]
 [0.41503649 0.73406449 0.47345004 0.33112608]],[[0.50142582 0.02288794 0.9867883  0.06646458]
 [0.71259711 0.73245141 0.96514089 0.75292946]
 [0.41503649 0.73406449 0.47345004 0.33112608]])

SyntaxError: invalid syntax (2319381822.py, line 1)

In [ ]:
tf.reduce_mean([[0, 1.0, 2.0],
                      [10.0, 11.0, 12.0],
                      [20.0, 21.0, 22.0],
                      [30.0, 31.0, 32.0]])